# Dynamic Programming, Monte Carlo and Temporal Difference Learning
The key idea of Dynamic Programming (DP) is to use the Bellman equation to decompose the value function into simpler parts, and then to iteratively solve these parts. DP methods require a complete and accurate model of the environment, which is often not available. Monte Carlo (MC) methods, on the other hand, do not require a model of the environment, and can learn directly from experience. Temporal Difference (TD) learning is a combination of the two, and can learn directly from experience like MC methods, but can also bootstrap like DP methods.

## Dynamic Programming
Considering that our goal is to find the optimal policy $\pi_*$, we can define the optimal value function $v_*$ as:

$$ 
\begin{equation}
v_*(s) = \max_{a \in \mathcal{A}} \left[ R(s, a) + \gamma \sum_{s'} p(s' \mid s, a) v_*(s') \right].
\end{equation}
$$

In order to compute the state-value $v_{\pi}(s)$ we have to perform some king od **policy evaluation**. The policy evaluation step computes the value function for a given policy $\pi$. The value function is defined as:

\begin{align}
v_{\pi}(s) &= \mathbb{E}_{\pi} \left[G_t \mid S_t = s \right] \\
 &= \mathbb{E}_{\pi} \left[R_{t+1} + \gamma G_{t+1} \mid S_t = s \right] \\
&= \mathbb{E}_{\pi} \left[R_{t+1} + \gamma v_{\pi}(S_{t+1}) \mid S_t = s \right] \\
&= \sum_a \pi(a \mid s) \sum_{s', r} p(s', r \mid s, a) \left[ r + \gamma v_{\pi}(s') \right],
\end{align}


where $\pi(a \mid s)$ is the probability of taking action $a$ in state $s$ under policy $\pi$, and $p(s', r \mid s, a)$ is the probability of transitioning to state $s'$ and receiving reward $r$ after taking action $a$ in state $s$.

We can approximate this optimal value function iteratively using the update rule:
$$
v_{k+1}(s) = \sum_a \pi(a \mid s) \sum_{s', r} p(s', r \mid s, a) \left[ r + \gamma v_k(s') \right].
$$

This  algorithm is called **Iterative Policy Evaluation**. It is guaranteed to converge to the true value function $v_{\pi}$ as $k \rightarrow \infty$. In procedural
form it can be written as:
$$
\begin{split}
& \text{Initialize } v(s), \text{ for all } s \in \mathcal{S}^+ \text{ arbitrarily except that } v(\texttt{terminal}) = 0 \\
& \text{Repeat} \\
& \quad \Delta \leftarrow 0 \\
& \quad \text{For each } s \in \mathcal{S} \\
& \quad \quad v \leftarrow v(s) \\
& \quad \quad v(s) \leftarrow \sum_a \pi(a \mid s) \sum_{s', r} p(s', r \mid s, a) \left[ r + \gamma v(s') \right] \\
& \quad \quad \Delta \leftarrow \max(\Delta, |v - v(s)|) \\
& \text{until } \Delta < \theta \text{ (a small positive number)}
\end{split}
$$       

### Policy Improvement
Once we have the value function, we can improve the policy by acting greedily with respect to it. The policy improvement theorem states that if two policies $\pi$ and $\pi'$ are such that $q_{\pi}(s, \pi'(s)) \geq v_{\pi}(s)$ for all $s \in \mathcal{S}$, then the policy $\pi'$ must be as good as, or better than, $\pi$. This means that if we act greedily with respect to the value function, we are guaranteed to improve the policy. The policy improvement algorithm is:
$$
\begin{equation}
\pi'(s) = \arg \max_a \left[ \sum_{s', r} p(s', r \mid s, a) \left[ r + \gamma v_*(s') \right] \right].
\end{equation}
$$

### Policy Iteration
The policy iteration algorithm alternates between policy evaluation and policy improvement. It is guaranteed to converge to the optimal policy and value function. In procedural form, it can be written as:
$$
\begin{split}
& \text{Initialize } \pi \text{ arbitrarily, e.g., } \pi(s) = \texttt{random}, \text{ for all } s \in \mathcal{S} \\
& \text{Repeat} \\
& \quad \text{Policy Evaluation} \\
& \quad \quad \text{Repeat} \\
& \quad \quad \quad \Delta \leftarrow 0 \\
& \quad \quad \quad \text{For each } s \in \mathcal{S} \\
& \quad \quad \quad \quad v \leftarrow v(s) \\
& \quad \quad \quad \quad v(s) \leftarrow \sum_a \pi(a \mid s) \sum_{s', r} p(s', r \mid s, a) \left[ r + \gamma v(s') \right] \\
& \quad \quad \quad \quad \Delta \leftarrow \max(\Delta, |v - v(s)|) \\
& \quad \quad \text{until } \Delta < \theta \text{ (a small positive number)} \\
& \quad \text{Policy Improvement} \\
& \quad \quad \text{policy-stable} \leftarrow \texttt{true} \\
& \quad \quad \text{For each } s \in \mathcal{S} \\
& \quad \quad \quad \text{old-action} \leftarrow \pi(s) \\
& \quad \quad \quad \pi(s) \leftarrow \arg \max_a \left[ \sum_{s', r} p(s', r \mid s, a) \left[ r + \gamma v(s') \right] \right] \\
& \quad \quad \quad \text{If old-action} \neq \pi(s), \text{ then policy-stable} \leftarrow \texttt{false} \\
& \quad \text{until policy-stable} \\
\end{split}
$$      

**Exercise 4.5:** Define policy iteration for action values. Give a complete algorithm for computing $q_*$, analogous to that on page 80 for computing $v_*$. 
 
**answer:**
As for state values, we will alternate between policy evaluation and policy improvement. The policy evaluation step will be the same as before, but now we will evaluate the action values instead of the state values. The policy improvement step will also be the same, but now we will act greedily with respect to the action values, i.e:
$$
\begin{equation}
\pi'(s) = \arg \max_a q_{\pi}(s, a).
\end{equation}
$$

The algorithm  to estimate the optimal action-value function $q_*$ is:
$$
\begin{split}
& \text{Initialize } Q(s, a), \text{ for all } s \in \mathcal{S}^+, a \in \mathcal{A}(s), \text{ arbitrarily except that } q(\texttt{terminal}, \cdot) = 0 \\
& \text{Repeat} \\
& \quad \text{Policy Evaluation} \\
& \quad \quad \text{Repeat} \\
& \quad \quad \quad \Delta \leftarrow 0 \\
& \quad \quad \quad \text{For each } s \in \mathcal{S}, a \in \mathcal{A}(s) \\
& \quad \quad \quad \quad q \leftarrow Q(s, a) \\
& \quad \quad \quad \quad Q(s, a) \leftarrow \sum_{s', r} p(s', r \mid s, a) \left[ r + \gamma \max_{a'} Q(s', a') \right] \\
& \quad \quad \quad \quad \Delta \leftarrow \max(\Delta, |q - Q(s, a)|) \\
& \quad \quad \text{until } \Delta < \theta \text{ (a small positive number)} \\
& \quad \text{Policy Improvement} \\
& \quad \quad \text{policy-stable} \leftarrow \texttt{true} \\
& \quad \quad \text{For each } s \in \mathcal{S} \\
& \quad \quad \quad \text{old-action} \leftarrow \pi(s) \\
& \quad \quad \quad \pi(s) \leftarrow \arg \max_a Q(s, a) \\
& \quad \quad \quad \text{If old-action} \neq \pi(s), \text{ then policy-stable} \leftarrow \texttt{false} \\
& \quad \text{until policy-stable} \\
\end{split}
$$                            



                                       

### Value Iteration
The drawback of policy iteration is that it requires a complete policy evaluation at each step, which can be computationally expensive. But we can avoid this by performing only one sweep of policy evaluation, and then immediately improving the policy.

The value iteration algorithm is an alternative to policy iteration that uses a single sweep of policy evaluation to improve the policy. It is guaranteed to converge to the optimal policy and value function. In procedural form, it can be written as:

$$
\begin{equation}
v_{k+1}(s) = \max_a \sum_{s', r} p(s', r \mid s, a) \left[ r + \gamma v_k(s') \right].
\end{equation}
$$
 
Value iteration is a simpler algorithm that combines policy evaluation and policy improvement into a single step. It is guaranteed to converge to the optimal policy and value function. In procedural form, it can be written as:

$$
\begin{split}
& \text{Initialize } v(s), \text{ for all } s \in \mathcal{S}^+ \text{ arbitrarily except that } v(\texttt{terminal}) = 0 \\
& \text{Repeat} \\
& \quad \Delta \leftarrow 0 \\
& \quad \text{For each } s \in \mathcal{S} \\
& \quad \quad v \leftarrow v(s) \\
& \quad \quad v(s) \leftarrow \max_a \left[ \sum_{s', r} p(s', r \mid s, a) \left[ r + \gamma v(s') \right] \right] \\
& \quad \quad \Delta \leftarrow \max(\Delta, |v - v(s)|) \\
& \text{until } \Delta < \theta \text{ (a small positive number)} \\
& \text{Output a deterministic policy } \pi \text{ such that} \\
& \quad \pi(s) = \arg \max_a \left[ \sum_{s', r} p(s', r \mid s, a) \left[ r + \gamma v(s') \right] \right].
\end{split}
$$

### Example: The Gambler's Problem
The Gambler's Problem is a simple problem where the agent can bet on a coin flip. The agent wins the amount he bets with probability $p$, and loses the amount he bets with probability $1-p$. The agent's goal is to reach a target amount of money, and the state space is the agent's current amount of money. The agent can bet any amount of money up to the current amount, but not more. The reward is 0 on all transitions except those on which the gambler reaches his goal, when it is +1. The optimal policy is to bet the smallest amount of money that will reach the goal with probability 1. The optimal value function is:
$$
v_*(s) = \max_{a} \left[ p v_*(s + a) + (1-p) v_*(s - a) \right].
$$

The optimal policy is:
$$
\pi_*(s) = \arg \max_a \left[ p v_*(s + a) + (1-p) v_*(s - a) \right].
$$


## Monte Carlo Methods
Monte Carlo methods learn directly from experience, by averaging sample returns. They do not require a model of the environment, and can learn from sample episodes. The key idea is to estimate the value function by averaging the returns that have been observed from each state. The update rule is:
$$
v(s) \leftarrow v(s) + \alpha \left[ R + \gamma \max_{a'} v(s') - v(s) \right].
$$


## Q-learning
In value-iteration, we assume that we have a model of the environment, and we can compute the value of each state-action pair. But in practice, we often do not have a model of the environment, and we must learn from experience. In real-life finite games sometimes it is not possible to iterate over all possible states and actions. For example, Let's calculate the statespace pf atari games: The resolution of the screen is 210x160 pixels, and every pixel can have 128 colors. so the total number of possible screens is $128^{210x160}=128^{33600}$. Moreover 99.9\% of these screens are not valid screens for a game, so iterating over all of them would be mostly a waste of time, if feasible. Q-learning is a model-free TD control algorithm that can learn directly from experience.
 
Q-learning defines an algorithm to approximate $q_*$, the *optimal action-value function*. We can write it independently of the policy being followed, and it is guaranteed to converge to $q_*$ as long as all **state-action pairs** are visited infinitely often and the policy converges in the limit to the greedy policy.

\begin{equation}
Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha \left[ R_{t+1} + \gamma \max_a Q(S_{t+1}, a) - Q(S_t, A_t) \right].
\end{equation}

In procedural form, the algorithm is:

\begin{equation}
\begin{split}
& \text{Initialize } Q(s, a), \text{ for all } s \in \mathcal{S}^+, a \in \mathcal{A}(s), \text{ arbitrarily except that } Q(\texttt{terminal}, \cdot) = 0 \\
& \text{Repeat (for each episode):} \\
& \quad \text{Initialize } S \\
& \quad \text{Repeat (for each step of episode):} \\
& \quad \quad \text{Choose } A \text{ from } S \text{ using policy derived from } Q \text{ (e.g., } \epsilon\text{-greedy)} \\
& \quad \quad \text{Take action } A, \text{ observe } R, S' \\
& \quad \quad Q(S, A) \leftarrow Q(S, A) + \alpha \left[ R + \gamma \max_a Q(S', a) - Q(S, A) \right] \\
& \quad \quad S \leftarrow S' \\
& \quad \text{until } S \text{ is terminal}
\end{split}
\end{equation}

Q-learning is called an **off-policy** because it directly approximates $q_*$, the optimal action-value function, independent of the policy being followed. 

## SARSA
SARSA is an **on-policy** TD control algorithm. It is very similar to Q-learning, but instead of using the greedy policy to select the next action, it uses the same policy that is being learned about. The update rule is:

\begin{equation}
Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha \left[ R_{t+1} + \gamma Q(S_{t+1}, A_{t+1}) - Q(S_t, A_t) \right].
\end{equation}

In procedural form, the algorithm is:
\begin{equation}
\begin{split}
& \text{Initialize } Q(s, a), \text{ for all } s \in \mathcal{S}^+, a \in \mathcal{A}(s), \text{ arbitrarily except that } Q(\texttt{terminal}, \cdot) = 0 \\
& \text{Repeat (for each episode):} \\
& \quad \text{Initialize } S \\
& \quad \text{Choose } A \text{ from } S \text{ using policy derived from } Q \text{ (e.g., } \epsilon\text{-greedy)} \\
& \quad \text{Repeat (for each step of episode):} \\
& \quad \quad \text{Take action } A, \text{ observe } R, S', A' \\
& \quad \quad Q(S, A) \leftarrow Q(S, A) + \alpha \left[ R + \gamma Q(S', A') - Q(S, A) \right] \\
& \quad \quad S \leftarrow S'; A \leftarrow A' \\
& \quad \text{until } S \text{ is terminal}
\end{split}
\end{equation}
 The main difference of SARSA from Q-learning is that the choice action $A'$ is based on policy $\pi$, instead of the greedy action $\max_a Q(S', a)$.
**Example 6.6: Cliff Walking**
Run this example described on page 132 of the book.


## Expected SARSA
Expected SARSA is an alternative to Q-learning that is more robust to noise and variance in the estimates of the action values. It is defined as:

\begin{align}
Q(S_t, A_t) &\leftarrow Q(S_t, A_t) + \alpha \left[ R_{t+1} + \gamma \mathbb{E}_\pi \left[ Q(S_{t+1}, A_{t+1}) \mid S_{t+1} \right] - Q(S_t, A_t) \right]\\
&\leftarrow Q(S_t, A_t) + \alpha \left[ R_{t+1} + \gamma \sum_a \pi(a \mid S_{t+1}) Q(S_{t+1}, a) - Q(S_t, A_t) \right].
\end{align}

The above update rule, **uses the expected value instead of the maximum value over the next state,action pairs**. This is useful when the action values are noisy, and the maximum value may not be representative of the true value of the action. The expected value is more robust to noise, and is less likely to overestimate the action values. Otherwise this rule is very similar to Q-learning, updating at every time step. 